# Olasılık ve İstatistiğe Giriş
Bu not defterinde, daha önce tartıştığımız bazı kavramlarla oynayacağız. Olasılık ve istatistiğin birçok kavramı, Python'da veri işleme için kullanılan `numpy` ve `pandas` gibi büyük kütüphanelerde iyi bir şekilde temsil edilmektedir.


In [ ]:
import numpy as np
import pandas as pd
import random
import matplotlib.pyplot as plt

## Rastgele Değişkenler ve Dağılımlar
0 ile 9 arasında uniform bir dağılımdan 30 değerlik bir örneklem çizmeyle başlayalım. Ayrıca ortalama ve varyansı da hesaplayacağız.


In [ ]:
sample = [ random.randint(0,10) for _ in range(30) ]
print(f"Sample: {sample}")
print(f"Mean = {np.mean(sample)}")
print(f"Variance = {np.var(sample)}")

Örnekte kaç farklı değer olduğunu görsel olarak tahmin etmek için **histogram** çizebiliriz:


In [ ]:
plt.hist(sample)
plt.show()

## Gerçek Verilerin Analizi

Ortalama ve varyans, gerçek dünya verilerini incelerken çok önemlidir. [SOCR MLB Height/Weight Data](http://wiki.stat.ucla.edu/socr/index.php/SOCR_Data_MLB_HeightsWeights) sitesinden beyzbol oyuncularına ait verileri yükleyelim.


In [ ]:
df = pd.read_csv("../../data/SOCR_MLB.tsv",sep='\t', header=None, names=['Name','Team','Role','Weight','Height','Age'])
df


> Burada veri analizi için [**Pandas**](https://pandas.pydata.org/) adlı bir paket kullanıyoruz. Pandas ve Python'da veri ile çalışma konusu kursun ilerleyen bölümlerinde daha ayrıntılı ele alınacaktır.

Yaş, boy ve kilo için ortalama değerleri hesaplayalım:


In [ ]:
df[['Age','Height','Weight']].mean()

Şimdi yükseklik üzerine odaklanalım ve standart sapma ile varyansı hesaplayalım:


In [ ]:
print(list(df['Height'])[:20])

In [ ]:
mean = df['Height'].mean()
var = df['Height'].var()
std = df['Height'].std()
print(f"Mean = {mean}\nVariance = {var}\nStandard Deviation = {std}")

Ortalamanın yanı sıra, medyan değeri ve çeyrek değerlerine bakmak da mantıklıdır. Bunlar bir **kutu grafiği** kullanılarak görselleştirilebilir:


In [ ]:
plt.figure(figsize=(10,2))
plt.boxplot(df['Height'].ffill(), vert=False, showmeans=True)
plt.grid(color='gray', linestyle='dotted')
plt.tight_layout()
plt.show()

Veri setimizin alt kümelerinin kutu grafikleri de oluşturabiliriz, örneğin, oyuncu rolüne göre gruplanmış olarak.


In [ ]:
df.boxplot(column='Height', by='Role', figsize=(10,8))
plt.xticks(rotation='vertical')
plt.tight_layout()
plt.show()

> **Not**: Bu diyagram, ortalama olarak birinci kale oyuncularının boylarının ikinci kale oyuncularının boylarından daha uzun olduğunu göstermektedir. Daha sonra, bu hipotezi daha resmi olarak nasıl test edebileceğimizi ve verilerimizin bunu göstermek için istatistiksel olarak anlamlı olduğunu nasıl kanıtlayabileceğimizi öğreneceğiz.  

Yaş, boy ve kilo sürekli rastgele değişkenlerdir. Sizce bunların dağılımı nedir? Bunu anlamanın iyi bir yolu değerlerin histogramını çizmektir: 


In [ ]:
df['Weight'].hist(bins=15, figsize=(10,6))
plt.suptitle('Weight distribution of MLB Players')
plt.xlabel('Weight')
plt.ylabel('Count')
plt.tight_layout()
plt.show()

## Normal Dağılım

Gerçek verilerimizle aynı ortalama ve varyansa sahip normal dağılımı takip eden yapay bir ağırlık örneği oluşturalım:


In [ ]:
generated = np.random.normal(mean, std, 1000)
generated[:20]

In [ ]:
plt.figure(figsize=(10,6))
plt.hist(generated, bins=15)
plt.tight_layout()
plt.show()

In [ ]:
plt.figure(figsize=(10,6))
plt.hist(np.random.normal(0,1,50000), bins=300)
plt.tight_layout()
plt.show()

Gerçek hayattaki değerlerin çoğu normal dağıldığı için, örnek veri oluşturmak için uniform rastgele sayı üreteci kullanmamalıyız. İşte uniform dağılımla ağırlık oluşturmaya çalışırsak ( `np.random.rand` tarafından oluşturulan) ne olur:


In [ ]:
wrong_sample = np.random.rand(1000)*2*std+mean-std
plt.figure(figsize=(10,6))
plt.hist(wrong_sample)
plt.tight_layout()
plt.show()

## Güven Aralıkları

Şimdi beyzbol oyuncularının kilo ve boyları için güven aralıklarını hesaplayalım. [Bu stackoverflow tartışmasından](https://stackoverflow.com/questions/15033511/compute-a-confidence-interval-from-sample-data) kodu kullanacağız:


In [ ]:
import scipy.stats

def mean_confidence_interval(data, confidence=0.95):
    a = 1.0 * np.array(data)
    n = len(a)
    m, se = np.mean(a), scipy.stats.sem(a)
    h = se * scipy.stats.t.ppf((1 + confidence) / 2., n-1)
    return m, h

for p in [0.85, 0.9, 0.95]:
    m, h = mean_confidence_interval(df['Weight'].fillna(method='pad'),p)
    print(f"p={p:.2f}, mean = {m:.2f} ± {h:.2f}")

## Hipotez Testi

Hadi beyzbol oyuncuları veri setimizde farklı rolleri inceleyelim:


In [ ]:
df.groupby('Role').agg({ 'Weight' : 'mean', 'Height' : 'mean', 'Age' : 'count'}).rename(columns={ 'Age' : 'Count'})

Birinci Kale Oyuncularının İkinci Kale Oyuncularından daha uzun olduğu hipotezini test edelim. Bunu yapmanın en basit yolu güven aralıklarını test etmektir:


In [ ]:
for p in [0.85,0.9,0.95]:
    m1, h1 = mean_confidence_interval(df.loc[df['Role']=='First_Baseman',['Height']],p)
    m2, h2 = mean_confidence_interval(df.loc[df['Role']=='Second_Baseman',['Height']],p)
    print(f'Conf={p:.2f}, 1st basemen height: {m1-h1[0]:.2f}..{m1+h1[0]:.2f}, 2nd basemen height: {m2-h2[0]:.2f}..{m2+h2[0]:.2f}')

Aralıkların çakışmadığını görebiliyoruz.

Hipotezi kanıtlamak için istatistiksel olarak daha doğru bir yol **Student t-testi** kullanmaktır:


In [ ]:
from scipy.stats import ttest_ind

tval, pval = ttest_ind(df.loc[df['Role']=='First_Baseman',['Height']], df.loc[df['Role']=='Second_Baseman',['Height']],equal_var=False)
print(f"T-value = {tval[0]:.2f}\nP-value: {pval[0]}")

`ttest_ind` fonksiyonunun döndürdüğü iki değer şunlardır:
* p-değeri, iki dağılımın aynı ortalamaya sahip olma olasılığı olarak düşünülebilir. Bizim durumumuzda, bu çok düşüktür, bu da ilk baz oyuncularının daha uzun olduğunu destekleyen güçlü kanıt olduğu anlamına gelir.
* t-değeri, t-testinde kullanılan normalize edilmiş ortalama farkının ara değeridir ve belirli bir güven değeri için eşik değerle karşılaştırılır.


## Merkezi Limit Teoremi ile Normal Dağılım Simülasyonu

Python'daki sahte rasgele sayı üreteci, bize uniform dağılım sağlaması için tasarlanmıştır. Normal dağılım için bir üreteç oluşturmak istiyorsak, merkezi limit teoremini kullanabiliriz. Normal dağılımlı bir değer elde etmek için, uniform dağılım kullanılarak oluşturulan bir örneklemin ortalamasını hesaplarız.


In [ ]:
def normal_random(sample_size=100):
    sample = [random.uniform(0,1) for _ in range(sample_size) ]
    return sum(sample)/sample_size

sample = [normal_random() for _ in range(100)]
plt.figure(figsize=(10,6))
plt.hist(sample)
plt.tight_layout()
plt.show()

## Korelasyon ve Kötü Niyetli Beyzbol Şirketi

Korelasyon, veri dizileri arasındaki ilişkileri bulmamıza olanak tanır. Oyuncak örneğimizde, kötücül bir beyzbol şirketinin oyuncularına boylarına göre ödeme yaptığını varsayalım - oyuncu ne kadar uzunsa, o kadar çok para kazanır. Diyelim ki bir taban maaş 1000 $ ve boy uzunluğuna bağlı olarak 0 ile 100 $ arasında ek bir bonus vardır. MLB'den gerçek oyuncuları alacağız ve hayali maaşlarını hesaplayacağız:


In [ ]:
heights = df['Height'].fillna(method='pad')
salaries = 1000+(heights-heights.min())/(heights.max()-heights.mean())*100
print(list(zip(heights, salaries))[:10])

Şimdi bu dizilerin kovaryansını ve korelasyonunu hesaplayalım. `np.cov` bize çoklu değişkenlere genişletilmiş olan **kovaryans matrisini** verecektir. Kovaryans matrisi $M$'nin elemanı $M_{ij}$, giriş değişkenleri $X_i$ ve $X_j$ arasındaki bir korelasyondur ve çapraz değerler $M_{ii}$, $X_{i}$'nin varyansıdır. Benzer şekilde, `np.corrcoef` bize **korelasyon matrisini** verecektir.


In [ ]:
print(f"Covariance matrix:\n{np.cov(heights, salaries)}")
print(f"Covariance = {np.cov(heights, salaries)[0,1]}")
print(f"Correlation = {np.corrcoef(heights, salaries)[0,1]}")

Bir korelasyonun 1'e eşit olması, iki değişken arasında güçlü bir **doğrusal ilişki** olduğu anlamına gelir. Doğrusal ilişkiyi, bir değeri diğerine karşı çizerek görsel olarak görebiliriz:


In [ ]:
plt.figure(figsize=(10,6))
plt.scatter(heights,salaries)
plt.tight_layout()
plt.show()

İlişki doğrusal değilse ne olur, buna bakalım. Diyelim ki şirketimiz, boylar ile maaşlar arasındaki bariz doğrusal bağımlılığı gizlemeye karar verdi ve formüle `sin` gibi bazı doğrusal olmayanlıklar ekledi:


In [ ]:
salaries = 1000+np.sin((heights-heights.min())/(heights.max()-heights.mean()))*100
print(f"Correlation = {np.corrcoef(heights, salaries)[0,1]}")

Bu durumda, korelasyon biraz daha küçük, ancak yine de oldukça yüksek. Şimdi, ilişkiyi daha az belirgin hale getirmek için maaşa bazı rastgele değişkenler ekleyerek ekstra rastgelelik katmak isteyebiliriz. Bakalım ne olacak:


In [ ]:
salaries = 1000+np.sin((heights-heights.min())/(heights.max()-heights.mean()))*100+np.random.random(size=len(heights))*20-10
print(f"Correlation = {np.corrcoef(heights, salaries)[0,1]}")

In [ ]:
plt.figure(figsize=(10,6))
plt.scatter(heights, salaries)
plt.tight_layout()
plt.show()

> Noktaların neden bu şekilde dikey çizgiler halinde sıralandığını tahmin edebilir misiniz?

Maaş gibi yapay olarak tasarlanmış bir kavram ile gözlemlenen değişken *boy* arasındaki korelasyonu gözlemledik. Şimdi de boy ve kilo gibi iki gözlemlenen değişkenin birbirleriyle korelasyon gösterip göstermediğine bakalım:


In [ ]:
np.corrcoef(df['Height'].ffill(),df['Weight'])

Maalesef, herhangi bir sonuç alamadık - sadece bazı garip `nan` değerler. Bu, serimizdeki bazı değerlerin tanımsız olması, `nan` olarak temsil edilmesi ve bunun işlemin sonucunun da tanımsız olmasına neden olmasıdır. Matrikse baktığımızda, `Weight` sütununun problemin kaynağı olduğunu görebiliriz, çünkü `Height` değerleri arasındaki kendi kendine korelasyon hesaplanmıştır.

> Bu örnek, **veri hazırlama** ve **temizleme**nin önemini göstermektedir. Doğru veriler olmadan hiçbir şey hesaplayamayız.

Eksik değerleri doldurmak için `fillna` metodunu kullanalım ve korelasyonu hesaplayalım: 


In [ ]:
np.corrcoef(df['Height'].fillna(method='pad'), df['Weight'])

Gerçekten bir korelasyon vardır, ancak bizim yapay örneğimizdeki kadar güçlü değildir. Gerçekten de, bir değeri diğerine karşı gösteren dağılım grafiğine bakarsak, ilişki çok daha az belirgin olacaktır:


In [ ]:
plt.figure(figsize=(10,6))
plt.scatter(df['Weight'],df['Height'])
plt.xlabel('Weight')
plt.ylabel('Height')
plt.tight_layout()
plt.show()

## Sonuç

Bu not defterinde, istatistiksel fonksiyonları hesaplamak için veri üzerinde temel işlemlerin nasıl yapılacağını öğrendik. Artık bazı hipotezleri kanıtlamak için sağlam bir matematik ve istatistik düzenini nasıl kullanacağımızı ve veri örneği verilmiş rastgele değişkenler için güven aralıklarının nasıl hesaplanacağını biliyoruz.


---

<!-- CO-OP TRANSLATOR DISCLAIMER START -->
**Feragatname**:  
Bu belge, AI çeviri hizmeti [Co-op Translator](https://github.com/Azure/co-op-translator) kullanılarak çevrilmiştir. Doğruluk için çaba göstersek de, otomatik çevirilerin hatalar veya yanlışlıklar içerebileceğini lütfen unutmayın. Orijinal belge, kendi dilinde yetkili kaynak olarak kabul edilmelidir. Kritik bilgiler için profesyonel insan çevirisi önerilir. Bu çevirinin kullanımı sonucu oluşabilecek yanlış anlamalar veya yanlış yorumlardan sorumlu tutulmayız.
<!-- CO-OP TRANSLATOR DISCLAIMER END -->
